# 🎬 Camera (and Director)

The `GraphCamera` and `GraphDirector` and [behaviors](./Behaviors.ipynb) observe and
control the current contents of the viewport. While the 2D and 3D APIs are very similar,
there are some subtle differences.

In [ ]:
if __name__ == "__main__" and "pyodide" in __import__("sys").modules:
    %pip install -q -r requirements.txt

In [ ]:
import json
from pathlib import Path

import ipyforcegraph.behaviors as B
import ipyforcegraph.graphs as G
import ipylab as L
import ipywidgets as W
import traitlets as T

## Load Data

In [ ]:
data = json.loads(Path("./datasets/blocks.json").read_text())

## Create Graphs and Behaviors

A `GraphCamera` generally can't be shared between two views of even the same
`ForceGraph` (e.g. from _Create New View for Output_) much less different graphs.

Similarly, the `center` of a `GraphDirector` won't work very well. The data-driven
`visible`, which can be a truthy `Column` or `Nunjucks` template, will generally work,
however.

In [ ]:
c2 = B.GraphCamera(capturing=True)
d2 = B.GraphDirector()
fg2 = G.ForceGraph(behaviors=[c2, d2])
fg2.source.nodes, fg2.source.links = data["nodes"], data["links"]

In [ ]:
c3 = B.GraphCamera(capturing=True)
d3 = B.GraphDirector()
fg3 = G.ForceGraph3D(behaviors=[c3, d3])
fg3.source.nodes, fg3.source.links = data["nodes"], data["links"]

## Create Controls

In [ ]:
graph_toggle = W.SelectionSlider(options={"2d": fg2, "3d": fg3})

A number of controls need some dimensions.

In [ ]:
xyz = "xyz"
min_max = dict(min=-3000, max=3000)
k_min_max = dict(min=0, max=5, step=0.01)
dis = dict(disabled=True)
dis_min_max = dict(**dis, **min_max)

`GraphCamera` instances observing different graphs provide different view data, such as
`zoom` in a 2D graph and `look_at` in a 3D graph.

In [ ]:
c_kxyz = {x: W.FloatSlider(description=x, **dis_min_max) for x in xyz}
c_kxyz["k"] = W.FloatSlider(description="zoom", **k_min_max, **dis)
c_capture = W.ToggleButton(description="capture", icon="crop")
c_vis = W.IntText(description="visible", **dis)
l_label = W.Label("look at")
l_xyz = {x: W.FloatSlider(description=x, **dis_min_max) for x in xyz}

These are mostly shared by the `GraphDirector`.

In [ ]:
d_kxyz = {x: W.FloatSlider(description=x, **min_max) for x in xyz}
d_kxyz["k"] = W.FloatSlider(description="zoom", **k_min_max)
dl_label = W.Label("look at")
dl_xyz = {x: W.FloatSlider(description=x, **min_max) for x in xyz}

Changing the `center`, `zoom`, or `look_at` have durations, expressed in seconds, and
the position is continuously reported by the `GraphCamera`: this is why they are two
separate behaviors.

In [ ]:
durs = ["pan", "zoom"]
d_dur = {dur: W.FloatSlider(description=f"{dur} (s)", max=5) for dur in durs}
btn_action = W.Button(description="action", icon="play", button_style="success")
btn_follow = W.ToggleButton(description="follow", icon="lock", value=True)

Fitting `visible` nodes can be accomplished with a `Nunjucks`: any `node` expression
that evaluates to a truthy value will be included in the eventual bound box (or 3d
_frustum_). Additionally _padding_ can be applied to leave a little more space.

In [ ]:
users = {d["user"] for d in data["nodes"] if "user" in d}
sel_user = W.Dropdown(description="by user", options=sorted(users))
tmpl_txt = W.Textarea(description="template")
tmpl_nj = B.Nunjucks("")
fit_pad = W.FloatSlider(description="padding", min=0, max=200)
btn_tmpl = W.ToggleButton(description="use template", icon="filter")

## Handle Camera Events

In [ ]:
def on_zoomed(*_):
    if graph_toggle.value == fg2:
        l_label.layout.display = dl_label.layout.display = "none"
        c_kxyz["k"].layout.display = "flex"
        c_kxyz["k"].value = c2.zoom
        if btn_follow.value:
            d_kxyz["k"].value = c2.zoom
        for i, x in enumerate(xyz):
            l_xyz[x].layout.display = dl_xyz[x].layout.display = "none"
            if c2.center and x != "z":
                c_kxyz[x].value = c2.center[i]
                if btn_follow.value:
                    d_kxyz[x].value = c2.center[i]
    else:
        c_kxyz["k"].layout.display = d_kxyz["k"].layout.display = "none"
        l_label.layout.display = dl_label.layout.display = "flex"
        for i, x in enumerate(xyz):
            l_xyz[x].layout.display = dl_xyz[x].layout.display = "flex"
            if c3.center and c3.look_at:
                c_kxyz[x].value, l_xyz[x].value = c3.center[i], c3.look_at[i]
                if btn_follow.value:
                    d_kxyz[x].value, dl_xyz[x].value = c3.center[i], c3.look_at[i]


def on_visible(*_):
    c_vis.value = len(c2.visible) if graph_toggle.value == fg2 else len(c3.visible)

## Connect the Camera

In [ ]:
graph_toggle.observe(on_zoomed, "value")
btn_follow.observe(on_zoomed, "value")
[c.observe(on_visible) for c in [c2, c3]]

T.link((c2, "capturing"), (c_capture, "value"))
T.link((c3, "capturing"), (c_capture, "value"))

c2.observe(on_zoomed, ["zoom", "center"])
c3.observe(on_zoomed, ["center", "look_at"])

## Connect the Director

In [ ]:
def on_direct(*_):
    if graph_toggle.value == fg2:
        with d2.hold_sync():
            d2.zoom = d_kxyz["k"].value
            d2.center = [d_kxyz[x].value for x in xyz]
        d2.send_state("zoom")
        d2.send_state("center")
    else:
        with d3.hold_sync():
            d3.center = [d_kxyz[x].value for x in xyz]
            d3.look_at = [dl_xyz[x].value for x in xyz]
        d3.send_state("look_at")
        d3.send_state("center")


[T.link((d2, f"{dur}_duration"), (ds, "value")) for dur, ds in d_dur.items()]
[T.link((d3, f"{dur}_duration"), (ds, "value")) for dur, ds in d_dur.items()]
btn_action.on_click(on_direct)

## Build Templates

In [ ]:
def on_tmpl(*_):
    if graph_toggle.value == fg2:
        d3.visible = ""
        if not btn_tmpl.value:
            d2.visible = ""
            return
        d2.visible = tmpl_nj
    else:
        d2.visible = ""
        if not btn_tmpl.value:
            d3.visible = ""
            return
        d3.visible = tmpl_nj


graph_toggle.observe(on_tmpl, "value")
T.dlink(
    (sel_user, "value"), (tmpl_txt, "value"), lambda x: "{{ node.user == '%s' }}" % x
)
T.link((d2, "padding"), (fit_pad, "value"))
T.link((d3, "padding"), (fit_pad, "value"))
T.dlink((tmpl_txt, "value"), (tmpl_nj, "value"))
btn_tmpl.observe(on_tmpl, "value")

## Build the UI

In [ ]:
ui = W.HBox(
    [
        W.VBox(
            [
                W.HBox([W.Label("camera"), graph_toggle]),
                *c_kxyz.values(),
                l_label,
                *l_xyz.values(),
                W.HBox([c_capture, c_vis]),
                W.Label("director"),
                W.Tab(
                    [
                        W.VBox(
                            [
                                *d_kxyz.values(),
                                dl_label,
                                *dl_xyz.values(),
                                *d_dur.values(),
                                W.HBox([btn_follow, btn_action]),
                            ]
                        ),
                        W.VBox([sel_user, tmpl_txt, fit_pad, btn_tmpl]),
                    ],
                    titles=["by center", "by nodes"],
                ),
            ],
            layout=dict(min_width="25em"),
        ),
        fg2,
        fg3,
    ],
    layout=dict(height="100%"),
)

## Make a Panel

In [ ]:
shell = L.JupyterFrontEnd().shell
panel = L.Panel([ui])
panel.title.label = "💡🎥🎬"
shell.add(panel, "main", {"mode": "split-right"})